# Finetuning using IndoBART

In [1]:
! git clone https://github.com/indobenchmark/indonlg.git

Cloning into 'indonlg'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 152 (delta 75), reused 97 (delta 34), pack-reused 0
Receiving objects: 100% (152/152), 2.32 MiB | 8.64 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [2]:
%cd /content/indonlg
! pip install -r requirements.txt

/content/indonlg
     |████████████████████████████████| 1.5 MB 11.4 MB/s 
     |████████████████████████████████| 68 kB 2.5 MB/s 
     |████████████████████████████████| 776.8 MB 18 kB/s 
     |████████████████████████████████| 186 kB 25.6 MB/s 
     |████████████████████████████████| 3.3 MB 39.1 MB/s 
     |████████████████████████████████| 394.7 MB 20 kB/s 
     |████████████████████████████████| 1.2 MB 43.4 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
     |████████████████████████████████| 65 kB 4.1 MB/s 
     |████████████████████████████████| 149 kB 53.3 MB/s 
     |████████████████████████████████| 1.8 MB 36.8 MB/s 
     |████████████████████████████████| 212 kB 52.9 MB/s 
     |████████████████████████████████| 136 kB 52.9 MB/s 
     |████████████████████████████████| 462 kB 47.0 MB/s 
     |████████████████████████████████| 2.9 MB 39.1 MB/s 
     |████████████████████████████████| 3.8 MB 37.2 MB/s 
     |████████████████████████████████| 132 kB 43.8 MB/s 
 

In [3]:
%cd /content/indonlg/examples

/content/indonlg/examples


In [4]:
import os, sys
sys.path.append('../')
os.chdir('../')

import torch
import shutil
import random
import numpy as np
import pandas as pd
from torch import optim
from transformers import MBartForConditionalGeneration

from indobenchmark import IndoNLGTokenizer
from utils.train_eval import train, evaluate
from utils.metrics import generation_metrics_fn
from utils.forward_fn import forward_generation
from utils.data_utils import MachineTranslationDataset, GenerationDataLoader

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
# Set random seed
# set_seed(26092020)

# Load Model

In [ ]:
bart_model = MBartForConditionalGeneration.from_pretrained('indobenchmark/indobart-v2')
tokenizer = IndoNLGTokenizer.from_pretrained('indobenchmark/indobart-v2')

model = bart_model
model

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(40004, 768, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(40004, 768, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 768, padding_idx=1)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=Tru

In [ ]:
count_param(model)

131543040

# Prepare Dataset

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = False
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8

source_lang = "[indonesian]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = './save/minang/example_min_id'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_path = '/content/drive/MyDrive/11-737/Assignment 2/FinalProject/data/min-id/train_preprocess.json'
valid_dataset_path = '/content/drive/MyDrive/11-737/Assignment 2/FinalProject/data/min-id/valid_preprocess.json'
test_dataset_path = '/content/drive/MyDrive/11-737/Assignment 2/FinalProject/data/min-id/test_preprocess.json'

train_dataset = MachineTranslationDataset(train_dataset_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset = MachineTranslationDataset(valid_dataset_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset = MachineTranslationDataset(test_dataset_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_loader = GenerationDataLoader(dataset=train_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

# Test model to generate sequences

In [ ]:
inputs = ['aku pergi ke toko obat membeli <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[indonesian]', decoder_lang_token='[indonesian]')

bart_input.to(device)
bart_out = model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> aku pergi ke toko obat membeli <mask> [indonesian]
<s> aku pergi ke toko obat membeli obat.


In [ ]:
inputs = ['aku menyang pasar karo <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[javanese]', decoder_lang_token='[javanese]')

bart_input.to(device)
bart_out = bart_model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> aku menyang pasar karo <mask> [javanese]
<s> aku menyang pasar karo tuku </s>


In [ ]:
inputs = ['kuring ka pasar senen meuli daging <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[sundanese]', decoder_lang_token='[sundanese]')

bart_input.to(device)
bart_out = bart_model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> kuring ka pasar senen meuli daging <mask> [sundanese]
<s> kuring ka pasar senen meuli daging sapi.


# Test model to translate

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 400/400 [09:52<00:00,  1.48s/it]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result.csv")

== Prediction Result ==
                                                 hyp  \
0  papar angan pena antar o kal ompok adat tah ad...   
1                                     o ch re ic eps   
2  n yo manusia jo mudah man ari mo prinsip du al...   
3  kode nan mang ara jo an tugas  <0x40> - <0x40>...   
4  panik ahan tu ind ak bar akh ia elok , sah ing...   

                                               label  
0  kehadiran & qu ot ; bo eka mata & qu ot ; meny...  
1  sop hr onica o ch re ic eps adalah spesies kum...  
2  umumnya manusia dengan mudah menerima prinsip ...  
3  bagian kode yang melakukan tugas  <0x40> - <0x...  
4  namun pernikahan tersebut tidak berakhir denga...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.000000   1.000000   1.000000
mean   28.355352  28.375633  42.393263  27.435855  42.076716  42.085392
std          NaN        NaN        NaN        NaN        NaN        

In [ ]:
metric_df

,BLEU,SacreBLEU,ROUGE1,ROUGE2,ROUGEL,ROUGELsum
0,28.355352,28.375633,42.393263,27.435855,42.076716,42.085392


# Fine Tuning & Evaluation

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:0.9919 LR:0.00010000: 100%|██████████| 1447/1447 [04:28<00:00,  5.40it/s]


(Epoch 1) TRAIN LOSS:0.9919 BLEU:73.98 SacreBLEU:74.62 ROUGE1:84.11 ROUGE2:72.57 ROUGEL:83.69 ROUGELsum:83.68 LR:0.00010000


VALID LOSS:0.6460: 100%|██████████| 200/200 [00:11<00:00, 17.69it/s]


(Epoch 1) VALID LOSS:0.6460 BLEU:79.08 SacreBLEU:79.09 ROUGE1:88.65 ROUGE2:79.83 ROUGEL:88.37 ROUGELsum:88.38


(Epoch 2) TRAIN LOSS:0.4641 LR:0.00009000: 100%|██████████| 1447/1447 [04:33<00:00,  5.30it/s]


...

(Epoch 9) VALID LOSS:0.7275 BLEU:81.57 SacreBLEU:81.57 ROUGE1:89.98 ROUGE2:82.32 ROUGEL:89.74 ROUGELsum:89.75
count stop: 3


(Epoch 10) TRAIN LOSS:0.0271 LR:0.00003874: 100%|██████████| 1447/1447 [04:32<00:00,  5.31it/s]


(Epoch 10) TRAIN LOSS:0.0271 BLEU:98.60 SacreBLEU:98.64 ROUGE1:99.51 ROUGE2:98.99 ROUGEL:99.51 ROUGELsum:99.51 LR:0.00003874


VALID LOSS:0.7327: 100%|██████████| 200/200 [00:11<00:00, 17.62it/s]


(Epoch 10) VALID LOSS:0.7327 BLEU:82.40 SacreBLEU:82.41 ROUGE1:90.29 ROUGE2:82.95 ROUGEL:90.05 ROUGELsum:90.08


In [ ]:
# Load best model
model.load_state_dict(torch.load(model_dir + "/best_model_0.th"))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 400/400 [05:58<00:00,  1.12it/s]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_jv-id.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_jv-id.csv")

== Prediction Result ==
                                                 hyp  \
0  kehadiran & qu ot ; bo eka mata & qu est ; men...   
1  sop hr onica o ch re ic eps adalah spesies kum...   
2  umumnya manusia dengan mudah menerima prinsip ...   
3  bagian kode yang mengerjakan tugas  <0x40> - <...   
4  namun pernikahan itu tidak berakhir baik , seh...   

                                               label  
0  kehadiran & qu ot ; bo eka mata & qu ot ; meny...  
1  sop hr onica o ch re ic eps adalah spesies kum...  
2  umumnya manusia dengan mudah menerima prinsip ...  
3  bagian kode yang melakukan tugas  <0x40> - <0x...  
4  namun pernikahan tersebut tidak berakhir denga...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1   ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.0000   1.000000   1.000000
mean   74.649488  74.661117  87.939626  79.8856  87.539399  87.527666
std          NaN        NaN        NaN      NaN        NaN        NaN
min 

In [ ]:
model_dir+"/best_model_0.th"

'./save/minang/example_min_id/best_model_0.th'

In [ ]:
! cp './save/minang/example_min_id/best_model_0.th' '/content/drive/My Drive/'